In [6]:
# end-to-end memory network

import numpy as np
from gensim import corpora
import math

def softmax(x):
    ex = np.exp(x - np.max(x))
    return ex / ex.sum()

def lj(j, J, k, d): 
    return (1-j/J)-(k/d)*(1-2*j/J)

if __name__=="__main__":
    # load input data
    #data = open('input1.txt', 'r').read() # input text file: sentences separated by .

    # hyperparameters
    d_embed = 10 # embedding dimension
    lr = 0.1 # learning rate
    
    # input to memory embedding mi = A * xi + tA
        #stoplist = set('for a of the and to in'.split())
    stoplist = []
    doc_raw = 'sam walks into the kitchen. sam picks up an apple. sam walks into the bedroom. sam drops the apple.'
    query_raw = 'where is sam'
    document = doc_raw.split('.')
    document.append(query_raw)
    senttoken = [ [word for word in sentence.lower().split(' ') if word not in stoplist] for sentence in document ]
    dictionary = corpora.Dictionary(senttoken)
    print(dictionary.token2id)
    document.pop(len(document)-1) # query at the end of document
    d_embed = len(dictionary)
    #voc = 0 # size of vocabulary
    #for d in document:
        #print(d)
        #if len(d) == 0:
            #document.remove(d)
        #voc = max(voc, len(d.split()))
    voc = d_embed
    n_memories = len(document)
    #print(voc)
    #print(document)

    # initiate weigth matrices
    A = np.random.randn(d_embed, voc)*0.1 # input to memory embedding
    tA = np.zeros(d_embed) # temporal encoding A
    B = np.random.randn(d_embed, voc)*0.1 # query embedding
    tB = np.zeros(d_embed) # temporal encoding B
    C = np.random.randn(d_embed, voc)*0.1 # output to memory embedding
    tC = np.zeros(d_embed) # temporal encoding C
    W = np.random.randn(voc, d_embed)*0.1 # final weight matrix
    
    # memory for Adagrad
    mA = np.zeros_like(A)
    mtA = np.zeros_like(tA)
    mB = np.zeros_like(B)
    mtB = np.zeros_like(tB)
    mC = np.zeros_like(C)
    mtC = np.zeros_like(tC)
    mW = np.zeros_like(W)
    
    for iterctr in range(10000):

        # forward pass

        # embedding simple: m_i = A_ij * x_ij + T_A_j
        x = np.zeros((n_memories, voc))
        m = np.zeros((n_memories, d_embed))
        for i in range(n_memories):
            thissent = document[i].split()
            for j in range(len(thissent)):
                x[i][j] = dictionary.token2id[thissent[j]]
        for i in range(n_memories):
            m[i] = np.dot(A, x[i].T) + tA # simple embedding
            #m[i][j] = lj(j,len(document[i]),j,d_embed) * A[i][j] * x[i][j] + tA[j] # with positional encoding
        #print(x)
        #print(m)

        # query embedding u = B * q + tB
        q = np.zeros(voc)
        u = np.zeros(d_embed)
        thissent = query_raw.split()
        for j in range(len(thissent)):
            q[j] = dictionary.token2id[thissent[j]]
        u = np.dot(B, q) + tB

        # match of query with memory p = softmax(u * mi) for all i
        p = np.zeros((n_memories, d_embed))
        p = softmax(np.dot(u, m.T))

        # output corresponding to input xi: ci = C * xi + tC
        c = np.zeros((n_memories, d_embed))
        for i in range(n_memories):
            c[i] = np.dot(C, x[i].T) + tC

        # response vector from memory o = sum pi * ci
        o = np.zeros(d_embed)
        o = np.dot(p.T, c)

        # predicted label a = softmax( W * (o + u))
        a_predict = softmax(np.dot(W, (o + u)))
        #print(a_predict)

        # backpropagation

        dA = np.zeros_like(A)
        dB = np.zeros_like(B)
        dC = np.zeros_like(C)
        dW = np.zeros_like(W)
        dtA = np.zeros_like(tA)
        dtB = np.zeros_like(tB)
        dtC = np.zeros_like(tC)

        truth = np.zeros_like(tA)
        truth[10] = 1 # bedroom
        dy = a_predict - truth
        # dA = dy a_predict * (1-a_predict) W sumi p[i] (1-p[i]) ( u.T * 1A * x[i]) c[i]
        #print('V: %d' % (voc))
        #print('d: %d', (d_embed))
        ABCunit = np.pad(np.identity(voc), ((0,d_embed-voc),(0,0)), 'constant', constant_values=(0))
        Wunit = np.pad(np.identity(voc), ((0,0), (0,d_embed-voc)), 'constant', constant_values=(0))
        tunit = np.ones_like(tA)

        dEAtmp = 0.
        for i in range(n_memories):
            dEAtmp += p[i]*(1.-p[i])*np.dot(np.dot(u, np.dot(ABCunit, x[i].T)), c[i])
        dEAtmp = W * dEAtmp
        dA = (np.dot(dy, a_predict * (1-a_predict)) * dEAtmp).T
        #print(dA)

        # dB = dy a_predict * (1-a_predict) W ( sumi p[i] (1-p[i]) ((1B q).T m[i]) c[i] + 1B q)
        dEAtmp = 0.
        for i in range(n_memories):
            dEAtmp += p[i]*(1.-p[i])*np.dot(np.dot(np.dot(ABCunit, q), m[i]), c[i])
        dEAtmp = W * dEAtmp
        dB = (np.dot(dy, a_predict*(1-a_predict)) * dEAtmp).T
        #print(dB)

        # dC = dy a_predict * (1-a_predict) W sumi p[i] 1C x[i]
        dEAtmp = 0.
        for i in range(n_memories):
            dEAtmp += p[i]*np.dot(ABCunit, x[i])
        dEAtmp = W * dEAtmp
        dC = (np.dot(dy, a_predict*(1-a_predict)) * dEAtmp).T
        #print(dC)

        # dW = dy a_predict * (1-a_predict) (o + u)
        dW = (np.dot(dy, a_predict*(1-a_predict)) * Wunit * (o + u))
        #print(dW)   

        # dtA = dy a_predict * (1-a_predict) W ( sumi p[i] (1-p[i]) (u.T 1tA) c[i])
        dEAtmp = 0.
        for i in range(n_memories):
            dEAtmp += p[i]*(1.-p[i])*np.dot(np.dot(u.T, tunit), c[i])
        dtA = (np.dot(dy * a_predict * (1-a_predict), W) * dEAtmp).T
        #print(dtA)

        # dtB = dy a_predict * (1-a_predict) W ( sumi p[i] (1-p[i]) ((1tB).T m[i] c[i]) + 1tB)
        dEAtmp = 0.
        for i in range(n_memories):
            dEAtmp += p[i]*(1.-p[i])*(np.dot(np.dot(tunit.T, m[i]), c[i]) + tunit)
        dtB = (np.dot(dy * a_predict * (1-a_predict), W) * dEAtmp).T
        #print(dtB)

        # dtC = dy a_predict * (1-a_predict) W ( sumi p[i] 1tC )
        dEAtmp = 0.
        for i in range(n_memories):
            dEAtmp += p[i]*tunit
        dtC = (np.dot(dy * a_predict * (1-a_predict), W) * dEAtmp).T
        #print(dtC)

        # maybe clip ?

        # update weights with Adagrad
        for weights, dweights, memwghts in zip([A,B,C,W,tA,tB,tC], [dA,dB,dC,dW,dtA,dtB,dtC], [mA,mB,mC,mW,mtA,mtB,mtC]):
            memwghts += dweights * dweights
            weights += -lr * dweights / np.sqrt(memwghts + 1.e-8)

        #print(A)
    print(a_predict)
    #print(np.argmax(a_predict))
    print(dictionary[np.argmax(a_predict)])

{'into': 0, 'kitchen': 1, 'sam': 2, 'the': 3, 'walks': 4, '': 5, 'an': 6, 'apple': 7, 'picks': 8, 'up': 9, 'bedroom': 10, 'drops': 11, 'is': 12, 'where': 13}
[1.31351085e-03 1.60122327e-07 7.42432013e-04 2.38615087e-03
 7.28085913e-06 1.34016459e-06 3.14062288e-05 7.39954981e-04
 7.87630566e-07 2.22681567e-03 9.90902630e-01 1.64277869e-03
 2.23272785e-07 4.52829329e-06]
bedroom
